In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Define image size and batch size
IMG_SIZE = 224
BATCH_SIZE = 32

In [6]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator=train_datagen.flow_from_directory(
    "/content/drive/MyDrive/brain/Brain_Tumor_Detection/Brain_Tumor_Detection/train",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/brain/Brain_Tumor_Detection/Brain_Tumor_Detection/train",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/brain/Brain_Tumor_Detection/Brain_Tumor_Detection/test',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)



Found 2400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Found 60 images belonging to 1 classes.


In [7]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
history = model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
75/75 [==============================] - 428s 6s/step - loss: 0.4844 - accuracy: 0.7892 - val_loss: 0.2773 - val_accuracy: 0.9033
Epoch 2/5
75/75 [==============================] - 331s 4s/step - loss: 0.2071 - accuracy: 0.9208 - val_loss: 0.1595 - val_accuracy: 0.9350
Epoch 3/5
75/75 [==============================] - 323s 4s/step - loss: 0.0954 - accuracy: 0.9733 - val_loss: 0.1084 - val_accuracy: 0.9550
Epoch 4/5
75/75 [==============================] - 325s 4s/step - loss: 0.0451 - accuracy: 0.9871 - val_loss: 0.0537 - val_accuracy: 0.9833
Epoch 5/5
75/75 [==============================] - 334s 4s/step - loss: 0.0109 - accuracy: 0.9992 - val_loss: 0.0085 - val_accuracy: 0.9983


In [12]:
model.save("Model.h5","label.txt")

In [13]:
#test your image
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

#load the model
model = load_model('/content/Model.h5')
#classes
class_names = ['You have Brain Tumor','You do not have Brain Tumor']
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

In [14]:
image = Image.open("/content/drive/MyDrive/brain/Brain_Tumor_Detection/Brain_Tumor_Detection/test/pred/pred37.jpg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

In [15]:
prediction = model.predict(data)
index = np.argmax(prediction)
confidence_score = prediction[0][index]
# Print prediction and confidence score
print("Result: ", class_names[index], end="")
print("\n")
print("Accuracy: ", confidence_score)

1/1 [==============================] - 0s 158ms/step
Result:  You have Brain Tumor

Accuracy:  1.0
